In [ ]:
%matplotlib inline
import os
import sys
import pylab as plt
import numpy as np
from glob import glob
import fitsio
from collections import Counter
from IPython.core.display import HTML

In [ ]:
from astrometry.util.fits import *
from astrometry.util.util import *
from astrometry.libkd.spherematch import match_radec
from tractor import *
from astrometry.util.starutil_numpy import *
from astrometry.util.plotutils import *

In [ ]:
C1 = fits_table('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/survey-ccds-90prime-dr9.kd.fits')
C2 = fits_table('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/survey-ccds-mosaic-dr9.kd.fits')
C3 = fits_table('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/survey-ccds-decam-dr9.kd.fits')

In [ ]:
B = fits_table('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/survey-bricks.fits.gz')

In [ ]:
Counter(C3.ccd_cuts)

In [ ]:
C3.width[0], C3.height[0]

In [ ]:
brad = np.sqrt(2) * 0.25 / 2.

In [ ]:
I1,J1,_ = match_radec(B.ra, B.dec, C1.ra, C1.dec, brad + 4096 * np.sqrt(2) / 2. * 0.454 / 3600)
I2,J2,_ = match_radec(B.ra, B.dec, C2.ra, C2.dec, brad + 4096 * np.sqrt(2) / 2. * 0.262 / 3600)
I3,J3,_ = match_radec(B.ra, B.dec, C3.ra, C3.dec, brad + np.hypot(4096, 2048) / 2. * 0.262 / 3600)

In [ ]:
II = np.unique(np.hstack((I1,I2,I3)))

In [ ]:
len(II) * 0.25**2

In [ ]:
brickarea = (B.dec2 - B.dec1) * (B.ra2 - B.ra1) * np.cos(np.deg2rad((B.dec1+B.dec2)/2.))

In [ ]:
np.sum(brickarea[II])

In [ ]:
#plt.plot(B.ra[II], B.dec[II], 'b.');

In [ ]:
W,H = 2000, 1000
ra_center = 120.
wcs = anwcs_create_allsky_hammer_aitoff2(ra_center, 0., W, H)

In [ ]:
bb = np.zeros((H,W), bool)
ok,xx,yy = wcs.radec2pixelxy(B.ra[II], B.dec[II])
print(Counter(ok))
xx = np.round(xx-1).astype(int)
yy = np.round(yy-1).astype(int)
xx.min(), yy.min(), xx.max(), yy.max()

In [ ]:
bb[yy,xx] = True
#plt.imshow(bb, origin='lower');

In [ ]:
A1 = fits_table('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/ccds-annotated-90prime-dr9.kd.fits')
A2 = fits_table('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/ccds-annotated-mosaic-dr9.kd.fits')
A3 = fits_table('/global/cfs/cdirs/cosmo/work/legacysurvey/dr9m/ccds-annotated-decam-dr9.kd.fits')

In [ ]:
A1.pixscale = 0.454
A2.pixscale = 0.262
A3.pixscale = 0.262

In [ ]:
A1.ra_lo = A1.ra_center - A1.width/2. * A1.pixscale / 3600. / np.cos(np.deg2rad(A1.dec_center))
A1.ra_hi = A1.ra_center + A1.width/2. * A1.pixscale / 3600. / np.cos(np.deg2rad(A1.dec_center))
A1.dec_lo = A1.dec_center - A1.height/2. * A1.pixscale / 3600.
A1.dec_hi = A1.dec_center + A1.height/2. * A1.pixscale / 3600.

A2.ra_lo = A2.ra_center - A2.width/2. * A2.pixscale / 3600. / np.cos(np.deg2rad(A2.dec_center))
A2.ra_hi = A2.ra_center + A2.width/2. * A2.pixscale / 3600. / np.cos(np.deg2rad(A2.dec_center))
A2.dec_lo = A2.dec_center - A2.height/2. * A2.pixscale / 3600.
A2.dec_hi = A2.dec_center + A2.height/2. * A2.pixscale / 3600.

A3.ra_lo = A3.ra_center - A3.height/2. * A3.pixscale / 3600. / np.cos(np.deg2rad(A3.dec_center))
A3.ra_hi = A3.ra_center + A3.height/2. * A3.pixscale / 3600. / np.cos(np.deg2rad(A3.dec_center))
A3.dec_lo = A3.dec_center - A3.width/2. * A3.pixscale / 3600.
A3.dec_hi = A3.dec_center + A3.width/2. * A3.pixscale / 3600.

A = merge_tables([A1,A2,A3])

In [ ]:
Counter(A.ra_lo < A.ra_hi), Counter(A.dec_lo < A.dec_hi)

In [ ]:
Counter(A.psfnorm_mean > 0)

In [ ]:
A.cut(A.psfnorm_mean > 0)

In [ ]:
A.psfiv = 1./(A.sig1 / A.psfnorm_mean)**2
A.galiv = 1./(A.sig1 / A.galnorm_mean)**2

In [ ]:
xx,yy = np.meshgrid(np.arange(W), np.arange(H))
ok,rr,dd = wcs.pixelxy2radec(xx, yy)
Counter(ok.flat)
xx = xx[ok]
yy = yy[ok]
rr = rr[ok]
dd = dd[ok]
len(dd)

In [ ]:
#A.radius = np.hypot(A.width, A.height) / 2. * A.pixscale / 3600.

In [ ]:
R = 0.454 * np.hypot(4096,4096)/2. / 3600.
R

In [ ]:
depths = {}
for band in 'grz':
    depth = np.zeros((H,W), np.float32)
    depths[band] = depth

    B = A[A.filter == band]
    print(len(B), 'in', band)
    I,J,_ = match_radec(rr, dd, B.ra, B.dec, R)
    print('Matched', len(I))
    K = np.flatnonzero((rr[I] > B.ra_lo[J]) * (rr[I] < B.ra_hi[J]) *
                       (dd[I] > B.dec_lo[J]) * (dd[I] < B.dec_hi[J]))
    I = I[K]
    J = J[K]
    print('Contained', len(I))
    for x,y,piv in zip(xx[I], yy[I], B.galiv[J]):
        depth[y,x] += piv

In [ ]:
from tractor.sfd import SFDMap
sfd = SFDMap()
filts = ['%s %s' % ('DES', f) for f in 'grz']
ebv,ext = sfd.extinction(filts, rr, dd, get_ebv=True)

In [ ]:
ext.shape

In [ ]:
for iband,band in enumerate('grz'):
    plt.figure(figsize=(12,8))
    plt.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.95)
    lo,hi = { 'g':(23.0,25.0), 'r':(22.4,24.4), 'z':(21.5,23.5) }[band]
    plt.clf()
    depth = -2.5 * (np.log10(5. * 1./np.sqrt(depths[band]))-9.)
    extmap = np.zeros_like(depth)
    extmap[yy,xx] = ext[:,iband]
    depth -= extmap
    plt.imshow(depth, origin='lower', vmin=lo, vmax=hi, cmap='RdYlBu')
    c = plt.colorbar(orientation='horizontal')
    c.set_label('%s-band depth (mag)' % band)

    dec_lo = -70
        
    dec_gridlines = list(range(dec_lo, 90, 10))
    dec_gridlines_ras = np.arange(ra_center-180, ra_center+180, 1)
    ra_gridlines = range(0, 360, 30)
    ra_gridlines_decs = np.arange(dec_lo, 90, 1.)
    for d in dec_gridlines:
        rr = dec_gridlines_ras
        dd = np.zeros_like(rr) + d
        ok,x,y = wcs.radec2pixelxy(rr, dd)
        plt.plot(x, y, 'k-', alpha=0.1)
    for r in ra_gridlines:
        dd = ra_gridlines_decs
        rr = np.zeros_like(dd) + r
        ok,x,y = wcs.radec2pixelxy(rr, dd)
        plt.plot(x, y, 'k-', alpha=0.1)

    ra_gridlines2 = [ra_center-180, ra_center+180]
    ra_gridlines2_decs = np.arange(dec_lo, 91, 1.)
    for r in ra_gridlines2:
        dd = ra_gridlines2_decs
        rr = np.zeros_like(dd) + r
        ok,x,y = wcs.radec2pixelxy(rr, dd)
        plt.plot(x, y, 'k-', alpha=0.5)
    
    ra_labels = ra_gridlines
    dec_labels = dec_gridlines
    ra_labels_dec = -30
    dec_labels_ra = ra_center+180
    
    ok,xi,yi = wcs.radec2pixelxy(ra_labels, ra_labels_dec)
    for x,y,v in zip(xi, yi, ra_labels):
        plt.text(x, y, '%i'%(v%360), ha='center', va='top', alpha=0.5)
    ok,xi,yi = wcs.radec2pixelxy(dec_labels_ra, dec_labels)
    for x,y,v in zip(xi, yi, dec_labels):
        plt.text(x-20, y, '%+i'%v, ha='right', va='center', alpha=0.5)

    # Galactic plane
    ll = np.linspace(0., 360., 720)
    bb = np.zeros_like(ll)
    rr,dd = lbtoradec(ll, bb)
    ok,x,y = wcs.radec2pixelxy(rr, dd)
    # Plot segments that are above Dec=-30 and not discontinuous
    d = np.append([0], np.hypot(np.diff(x), np.diff(y)))
    ok = (d < 100)# * (dd > -30)
    istart = 0
    while istart < len(ok):
        while istart < len(ok) and ok[istart] == False:
            istart += 1
        iend = istart
        while iend < len(ok) and ok[iend] == True:
            iend += 1
        if iend != istart:
            #print('Plotting from', istart, 'to', iend, 'ok', ok[istart:iend])
            plt.plot(x[istart:iend], y[istart:iend], '-', color='0.6', lw=2)
        istart = iend

    # Label regions
    for r,d,n in [(30, 0, 'DES'),
                  (0, 20, 'DECaLS'),
                  (180, 10, 'DECaLS'),
                  (180, 50, 'MzLS+BASS')]:
        ok,x,y = wcs.radec2pixelxy(r, d)
        plt.text(x, y, n, fontsize=16, ha='center', va='center')
    
    plt.xticks([])
    plt.yticks([])
    #plt.axis('equal');
    ax = [0,W, 0.1*H, H]
    plt.axis(ax)
    plt.axis('equal')
    plt.axis(ax)
    plt.gca().set_frame_on(False)
    plt.savefig('depth-map-%s.png' % band)
    plt.show()